In [20]:
# Step 1: Import necessary libraries
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
import numpy as np


print(f"TensorFlow version: {tf.__version__}")


TensorFlow version: 2.18.0


In [21]:
# Step 2: Define model parameters
IMG_HEIGHT = 224
IMG_WIDTH = 224
CHANNELS = 3
BATCH_SIZE = 32
NUM_CLASSES = 1  # Binary classification
LEARNING_RATE = 0.001


In [22]:
# Step 3: Create the model architecture
def create_model(fine_tune_at=100):
    # Load the pre-trained MobileNetV2 model
    base_model = MobileNetV2(
        input_shape=(IMG_HEIGHT, IMG_WIDTH, CHANNELS),
        include_top=False,
        weights='imagenet'
    )
    
    # Freeze the base model layers
    base_model.trainable = False
    
    # Create the model architecture
    inputs = base_model.input
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(NUM_CLASSES, activation='sigmoid')(x)
    
    # Create the model
    model = Model(inputs=inputs, outputs=outputs)
    
    return model, base_model

# Create the model
model, base_model = create_model()

# Display model summary
model.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer_2[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,422,081 (9.24 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [23]:
# Step 4: Configure model compilation
def compile_model(model):
    model.compile(
        optimizer=Adam(learning_rate=LEARNING_RATE),
        loss='binary_crossentropy',
        metrics=[
            'accuracy',
            tf.keras.metrics.Precision(),
            tf.keras.metrics.Recall(),
            tf.keras.metrics.AUC()
        ]
    )

compile_model(model)


In [24]:
# Step 5: Set up callbacks for training
def create_callbacks(model_name="mobilenetv2_genimage"):
    # Checkpoint callback
    checkpoint = ModelCheckpoint(
        f'checkpoints/{model_name}_best.keras',  # Changed extension to .keras
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    )
    
    # Early stopping callback
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    )
    
    # Learning rate reduction callback
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=3,
        min_lr=1e-6,
        verbose=1
    )
    
    return [checkpoint, early_stopping, reduce_lr]

# Create callbacks
callbacks = create_callbacks()


In [25]:
# Step 6: Create utility functions for model analysis
def plot_training_history(history):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot accuracy
    ax1.plot(history.history['accuracy'])
    ax1.plot(history.history['val_accuracy'])
    ax1.set_title('Model Accuracy')
    ax1.set_ylabel('Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.legend(['Train', 'Validation'], loc='upper left')
    
    # Plot loss
    ax2.plot(history.history['loss'])
    ax2.plot(history.history['val_loss'])
    ax2.set_title('Model Loss')
    ax2.set_ylabel('Loss')
    ax2.set_xlabel('Epoch')
    ax2.legend(['Train', 'Validation'], loc='upper left')
    
    plt.tight_layout()
    plt.show()


In [26]:
# Step 7: Create function for model fine-tuning
def prepare_model_for_fine_tuning(model, base_model, num_layers_to_fine_tune=100):
    # Unfreeze the last num_layers_to_fine_tune layers
    base_model.trainable = True
    for layer in base_model.layers[:-num_layers_to_fine_tune]:
        layer.trainable = False
    
    # Recompile the model with a lower learning rate
    model.compile(
        optimizer=Adam(learning_rate=LEARNING_RATE/10),
        loss='binary_crossentropy',
        metrics=[
            'accuracy',
            tf.keras.metrics.Precision(),
            tf.keras.metrics.Recall(),
            tf.keras.metrics.AUC()
        ]
    )
    
    return model

# This will be used later after initial training


In [27]:
# Step 8: Create prediction and evaluation utilities
def create_prediction_utilities():
    from sklearn.metrics import classification_report, confusion_matrix
    import seaborn as sns
    
    def plot_confusion_matrix(y_true, y_pred):
        cm = confusion_matrix(y_true, y_pred.round())
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
        plt.title('Confusion Matrix')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.show()
    
    def print_classification_metrics(y_true, y_pred):
        print("\nClassification Report:")
        print(classification_report(y_true, y_pred.round()))
    
    return plot_confusion_matrix, print_classification_metrics

plot_confusion_matrix, print_classification_metrics = create_prediction_utilities()


In [30]:
# Step 9: Model architecture verification

def verify_model_architecture():
    print("Model architecture and trainable layers:")
    for layer in model.layers:
        print(f"{layer.name}: {layer.trainable}")

    # Calculate total parameters
    trainable_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])  # Use .shape instead of .get_shape()
    non_trainable_params = np.sum([np.prod(v.shape) for v in model.non_trainable_weights])  # Use .shape instead of .get_shape()

    print(f"\nTotal trainable parameters: {trainable_params:,}")
    print(f"Total non-trainable parameters: {non_trainable_params:,}")

# Run the function
verify_model_architecture()


Model architecture and trainable layers:
input_layer_2: False
Conv1: False
bn_Conv1: False
Conv1_relu: False
expanded_conv_depthwise: False
expanded_conv_depthwise_BN: False
expanded_conv_depthwise_relu: False
expanded_conv_project: False
expanded_conv_project_BN: False
block_1_expand: False
block_1_expand_BN: False
block_1_expand_relu: False
block_1_pad: False
block_1_depthwise: False
block_1_depthwise_BN: False
block_1_depthwise_relu: False
block_1_project: False
block_1_project_BN: False
block_2_expand: False
block_2_expand_BN: False
block_2_expand_relu: False
block_2_depthwise: False
block_2_depthwise_BN: False
block_2_depthwise_relu: False
block_2_project: False
block_2_project_BN: False
block_2_add: False
block_3_expand: False
block_3_expand_BN: False
block_3_expand_relu: False
block_3_pad: False
block_3_depthwise: False
block_3_depthwise_BN: False
block_3_depthwise_relu: False
block_3_project: False
block_3_project_BN: False
block_4_expand: False
block_4_expand_BN: False
block_4

In [31]:
# Step 10: Save model architecture and configuration
def save_model_config():
    # Save model architecture as JSON
    model_json = model.to_json()
    with open("model_architecture.json", "w") as json_file:
        json_file.write(model_json)
    
    # Save model configuration
    model_config = {
        "img_height": IMG_HEIGHT,
        "img_width": IMG_WIDTH,
        "channels": CHANNELS,
        "batch_size": BATCH_SIZE,
        "learning_rate": LEARNING_RATE,
        "num_classes": NUM_CLASSES
    }
    
    import json
    with open("model_config.json", "w") as config_file:
        json.dump(model_config, config_file, indent=4)

save_model_config()
